<a href="https://colab.research.google.com/github/dineshgumray/ActiveLearning/blob/main/PredictionEntropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install small-text[transformers]  # use "small-text" without "[transformers]" if you want to work on the CPU only
%pip install datasets

In [10]:
import datasets
import logging
import numpy as np


# disables the progress bar for notebooks: https://github.com/huggingface/datasets/issues/2651
datasets.logging.get_verbosity = lambda: logging.NOTSET

raw_dataset = datasets.load_dataset('banking77')
num_classes = np.unique(raw_dataset['train']['label']).shape[0]

print('First 10 training samples:\n')
for i in range(10):
    print(raw_dataset['train']['label'][i], ' ', raw_dataset['train']['text'][i])

First 10 training samples:

11   I am still waiting on my card?
11   What can I do if my card still hasn't arrived after 2 weeks?
11   I have been waiting over a week. Is the card still coming?
11   Can I track my card while it is in the process of delivery?
11   How do I know if I will get my card, or if it is lost?
11   When did you send me my new card?
11   Do you have info about the card on delivery?
11   What do I do if I still have not received my new card?
11   Does the package with my card have tracking?
11   I ordered my card but it still isn't here


In [11]:
import transformers
from transformers import AutoTokenizer

transformers.logging.get_verbosity = lambda: logging.NOTSET


transformer_model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(
    transformer_model_name
)

In [12]:
from small_text.integrations.transformers.datasets import TransformersDataset


def get_transformers_dataset(tokenizer, data, labels, max_length=60):

    data_out = []

    for i, doc in enumerate(data):
        encoded_dict = tokenizer.encode_plus(
            doc,
            add_special_tokens=True,
            padding='max_length',
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='pt',
            truncation='longest_first'
        )

        data_out.append((encoded_dict['input_ids'], encoded_dict['attention_mask'], labels[i]))

    return TransformersDataset(data_out)


train = get_transformers_dataset(tokenizer, raw_dataset['train']['text'], raw_dataset['train']['label'])
test = get_transformers_dataset(tokenizer, raw_dataset['test']['text'], raw_dataset['test']['label'])

In [13]:
from small_text.active_learner import PoolBasedActiveLearner

from small_text.initialization import random_initialization_balanced
from small_text.integrations.transformers import TransformerModelArguments
from small_text.integrations.transformers.classifiers.factories import TransformerBasedClassificationFactory
from small_text.query_strategies import PredictionEntropy
from small_text.query_strategies import LeastConfidence
from small_text.query_strategies import BreakingTies
from small_text.integrations.transformers import TransformerModelArguments


# simulates an initial labeling to warm-start the active learning process
def initialize_active_learner(active_learner, y_train):
    

    x_indices_initial = random_initialization_balanced(y_train, n_samples=1000)
    y_initial = y_train[x_indices_initial]

    active_learner.initialize_data(x_indices_initial, y_initial)

    return x_indices_initial

transformer_model = TransformerModelArguments(transformer_model_name)
clf_factory = TransformerBasedClassificationFactory(transformer_model, 
                                                    num_classes, 
                                                    kwargs=dict({
                                                                 'mini_batch_size': 32,
                                                                 'early_stopping_no_improvement': -1
                                                                }))
query_strategy = PredictionEntropy()

active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train)
labeled_indices = initialize_active_learner(active_learner, train.y)

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
f = open("/content/drive/MyDrive/PredictionEntropy_log.txt", "a")

In [15]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

num_queries = 5

def evaluate(active_learner, train, test):
    y_pred = active_learner.classifier.predict(train)
    y_pred_test = active_learner.classifier.predict(test)
    test_acc = accuracy_score(y_pred_test, test.y)
    train_acc = accuracy_score(y_pred, train.y)

    print("Train accuracy: {}".format(train_acc))
    print("Test accuracy: {}".format(test_acc))
    print("Train Report \n")
    print(classification_report(train.y, y_pred))
    print("Test Report \n")
    print(classification_report(test.y, y_pred_test))
    f.write("\n Train Report \n")
    f.write(classification_report(train.y, y_pred))
    f.write("\n Test Report \n")
    f.write(classification_report(test.y, y_pred_test))
    f.write("\n")
    f.write("Train accuracy: ")
    f.write(str(train_acc))
    f.write("\n")
    f.write("Test accuracy: ")
    f.write(str(test_acc))
    return test_acc


results = []
results.append(evaluate(active_learner, train[labeled_indices], test))

    
for i in range(num_queries):
    # ...where each iteration consists of labelling 1000 samples
    q_indices = active_learner.query(num_samples=1000)

    # Simulate user interaction here. Replace this for real-world usage.
    y = train.y[q_indices]

    # Return the labels for the current query to the active learner.
    active_learner.update(y)

    labeled_indices = np.concatenate([q_indices, labeled_indices])

    print('Iteration #{:d} ({} samples)'.format(i, len(labeled_indices)))
    f.write("\n############################################################\n")
    f.write("Iteration ")
    f.write(str(i+1))
    f.write("\t Samples ")
    f.write(str(len(labeled_indices)))
    results.append(evaluate(active_learner, train[labeled_indices], test))
    
f.close()

Train accuracy: 0.866
Test accuracy: 0.6454545454545455
Train Report 

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       0.92      0.79      0.85        14
           2       1.00      1.00      1.00        12
           3       1.00      1.00      1.00        13
           4       1.00      0.83      0.91        12
           5       0.77      1.00      0.87        17
           6       1.00      0.83      0.91        12
           7       1.00      0.64      0.78        14
           8       1.00      0.92      0.96        12
           9       0.63      1.00      0.77        12
          10       0.54      1.00      0.70        14
          11       1.00      0.93      0.96        14
          12       0.85      0.92      0.88        12
          13       0.92      0.92      0.92        12
          14       1.00      0.93      0.96        14
          15       0.92      1.00      0.96        12
          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

Iteration #0 (2000 samples)
Train accuracy: 0.941
Test accuracy: 0.7646103896103896
Train Report 

              precision    recall  f1-score   support

           0       0.82      0.93      0.87        15
           1       1.00      1.00      1.00        37
           2       0.96      1.00      0.98        22
           3       1.00      1.00      1.00        26
           4       0.97      1.00      0.99        35
           5       0.84      0.97      0.90        33
           6       0.93      1.00      0.96        25
           7       0.90      0.98      0.93        44
           8       0.94      0.94      0.94        31
           9       0.81      1.00      0.90        22
          10       1.00      0.95      0.97        19
          11       0.94      0.89      0.92        19
          12       1.00      0.85      0.92        20
          13       1.00      0.64      0.78        14
          14       1.00      0.93      0.96        27
          15       0.86      1.00   